In [2]:
import json
import fnmatch
import os
from tqdm import tqdm
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random
from sklearn.cross_validation import train_test_split
from sklearn import metrics
#Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

/Users/andy/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
train = pd.read_csv("/Users/andy/Desktop/2017學習數據分析黑客松/hack_question02.csv")
df = pd.read_csv('allall.csv')

/Users/andy/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
len(df)

2652720

In [9]:
# all_uid
uid = df['user_id'].value_counts().index

In [10]:
len(uid)

2953

In [11]:
# all_course_id
k=train['course_id'].value_counts()
classname=k.index

In [12]:
#切天數
df['date']=df['time'].str.split("T").str[0]
#切時間
df['time1']=df['time'].str.split("T").str[1]
df['time1']=df['time1'].str.split(".").str[0]

In [15]:
#把每個欄位匯入
num=1
fdic={}
u_dic={}
c_dic={}
dc_dic={}
vc_dic={}
at_dic={}
sc_dic={}
se_dic={}
pl_dic={}
sevi_dic={}
spva_dic={}

#training的所有user
for e_u in tqdm(uid):
    k=df[df['user_id']==e_u]['course_id'].value_counts().index
    #training裡面user的所有課程
    for each in k:
        dn=df[(df['user_id']==e_u) & (df['course_id']==each)]
        #user_id
        u_dic[num]=e_u
        #course_id
        c_dic[num]=each
        #datecount
        dc_dic[num]=len(dn['date'].value_counts())
        #video_count
        vc_dic[num]=len(dn['video_id'].value_counts())
        #session_count
        se_dic[num]=len(dn['session'].value_counts())
        #影片觀看次數
        a=0
        session = []
        for len_session in range(len(dn['session'].value_counts())):
            if dn['session'] not in session:
                session.append(dn['session'])
                a+=len(dn['video_id'].value_counts())
        sevi_dic[num] = a
        #pause_count
        pl_dic[num]=len(dn[dn['event_type']=='pause_video'])
        #speed_change_count
        sc_dic[num]=len(dn[dn['event_type']=='speed_change_video'])
        #speed_change_value
        speed = 0
        speed+=(dn['new_speed']-dn['old_speed']).sum()
        spva_dic[num]=speed
        
        #all_time
        total=0
        p=[]
        n=dn.index
        for a in n:
            if dn['event_type'][a]=='play_video':
                p.append(a)
            elif dn['event_type'][a]=='pause_video' and p!=[]:
                if dn['session'][a] == dn['session'][p[-1]]:
                    d1=datetime.datetime.strptime(dn['time1'][a],"%H:%M:%S")
                    d2=datetime.datetime.strptime(dn['time1'][p[-1]],"%H:%M:%S")
                    diff=relativedelta(d1,d2)
                    total+=int(diff.minutes)*60+int(diff.seconds)
                p=[]
        at_dic[num]=total                  
        num+=1
fdic['user_id']=u_dic
fdic['course_id']=c_dic
fdic['datecount']=dc_dic
fdic['videocount']=vc_dic
fdic['session_videocount']=se_dic
fdic['totaltime']=at_dic
fdic['sessioncount']=se_dic
fdic['pause_video_count']=pl_dic
fdic['speed_change_count']=sc_dic
fdic['speed_change_value']=spva_dic
fdf=pd.DataFrame.from_dict(fdic)

100%|██████████| 2953/2953 [17:09<00:00,  2.87it/s]


In [18]:
fdf.to_csv("finallfinall.csv")

In [376]:
#以該課程所有人中最多觀看影片數量最多的人為基準，來做影片完成率
c_id=fdf['course_id'].value_counts().index
courseperid={}
ddm={}
num=1
for each in c_id:
    courseperid[each]=fdf[fdf['course_id']==each]['videocount'].max()
courseperid
for a in range(1,len(fdf)+1):
    ddm[num]=fdf['videocount'][a]/courseperid[fdf['course_id'][a]]
    num+=1
fdic['ddm']=ddm
fdf=pd.DataFrame.from_dict(fdic)

/Users/kerry/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in long_scalars


In [377]:
fdf

,course_id,datecount,ddm,pause_video_count,session_videocount,sessioncount,speed_change_count,speed_change_value,totaltime,user_id,videocount
1,ORG4638/C8644/D1465,6,0.800000,2,4,8,0,0.0,32,113,4
2,ORG4638/C9081/D1465,7,0.000000,0,0,9,0,0.0,0,113,0
3,ORG4638/C5126/D1465,4,NaN,0,0,5,0,0.0,0,113,0
4,ORG4638/C4949/D5126,5,NaN,0,0,6,0,0.0,0,113,0
5,ORG4638/C9081/D1465,7,0.000000,0,0,5,0,0.0,0,133,0
6,ORG4638/C3685/D3685,2,NaN,0,0,2,0,0.0,0,133,0
7,ORG4638/C5126/D1465,3,NaN,0,0,3,0,0.0,0,5,0
8,ORG4638/C9081/D1465,4,0.000000,0,0,4,0,0.0,0,5,0
9,ORG4638/C9081/D1465,3,0.000000,0,0,2,0,0.0,0,47,0
10,ORG4638/C8644/D1465,2,0.000000,0,0,2,0,0.0,0,47,0


In [ ]:
dic={} 
with open("hack_training.csv", 'r') as f:  
    for line in f: 
        course=line.split(',')[0] 
        userid =line.split(',')[1] 
        if course not in dic: 
            dic[course]=[userid]
        else: 
            dic[course].append(userid)
dic.pop('course_id')
for a in tqdm(course_list[:62]):
    df1=df[df["course_id"]==a]
    video_total=len(df1["video_id"].value_counts())
    for users in dic[a]:
        df2=df1[(df1["user_id"]==int(users))]
        video=len(df2["video_id"].value_counts())
        video_complete_rate = video/video_total if video_total != 0 else 0

In [312]:
dic={} 
with open("hack_training.csv", 'r') as f:  
    for line in f: 
        course=line.split(',')[0] 
        userid =line.split(',')[1] 
        if course not in dic: 
            dic[course]=[userid]
        else: 
            dic[course].append(userid)
df['date_a'] = df['time'].str.split('T').str[0]
df['time_c'] = df['time'].str.split('T').str[1].str.split('.').str[0]
course_list=[]
for i in dic:
    course_list.append(i)
course_list

['ORG9784/C7351/D0468',
 'ORG4638/C2366/D8928',
 'ORG1525/C8152/D9981',
 'ORG4638/C1465/D0293',
 'ORG4638/C0417/D9981',
 'ORG5441/C7825/D2453',
 'ORG4638/C0627/D1937',
 'ORG0137/C9976/D2671',
 'ORG4638/C3075/D3524',
 'ORG4638/C8244/D6039',
 'ORG0137/C9976/D3704',
 'ORG4638/C6077/D3704',
 'ORG4638/C1543/D3524',
 'ORG0137/C3721/D9981',
 'ORG4638/C6848/D6039',
 'ORG4638/C3685/D0323',
 'ORG4638/C5431/D3524',
 'ORG4638/C5998/D4398',
 'ORG4638/C8644/D1465',
 'ORG3685/C7590/D6433',
 'ORG4638/C5998/D7477',
 'ORG1525/C7481/D4353',
 'ORG0323/C4724/D6492',
 'ORG9615/C1168/D6289',
 'ORG0137/C3721/D0468',
 'ORG4638/C4145/D6937',
 'ORG4638/C8226/D9752',
 'ORG6937/C7514/D0468',
 'ORG4638/C6063/D2453',
 'ORG4638/C1465/D1525',
 'ORG0137/C7590/D9615',
 'ORG4638/C9094/D8070',
 'ORG0137/C0417/D2453',
 'ORG4638/C2976/D5552',
 'ORG4638/C8850/D3524',
 'ORG4638/C5126/D1465',
 'ORG0323/C4386/D1168',
 'ORG4638/C8166/D6039',
 'ORG8070/C4574/D4353',
 'ORG2288/C5915/D4353',
 'ORG4638/C6521/D8070',
 'ORG3685/C3456/

In [318]:
course_list.remove('course_id')

In [319]:
dic.pop('course_id')

['user_id']

In [335]:
all_dic={}
all_id = 0


for a in (course_list[:62]):
    df1=df[df["course_id"]==a]
    
    video_total=len(df1["video_id"].value_counts())
#     print(video_total)
    for users in dic[a]:
#         print(users)
        df2=df1[(df1["user_id"]==int(users))]
#         print(df2['video_id'])
        video=len(df2["video_id"].value_counts())
        video_complete_rate = video/video_total if video_total != 0 else 0
        all_dic[all_id]={"user_id":users,"course_id":a,'video_complete_rate':video_complete_rate}
        all_id+=1

In [336]:
aaaa = pd.DataFrame(all_dic)

In [340]:
aaaa.T

,course_id,user_id,video_complete_rate
0,ORG9784/C7351/D0468,27,0
1,ORG9784/C7351/D0468,47,0
2,ORG9784/C7351/D0468,50,0
3,ORG9784/C7351/D0468,2115,0
4,ORG9784/C7351/D0468,3101,0
5,ORG9784/C7351/D0468,7865,0
6,ORG9784/C7351/D0468,8395,0
7,ORG9784/C7351/D0468,8545,0
8,ORG9784/C7351/D0468,11327,0
9,ORG9784/C7351/D0468,11744,0


In [338]:
len(fdf)

267

In [10]:
#log_count
usertime = []
for i in range(len(data1)):
    try:
        usertime.append((data1[i]['context']['user_id'],data1[i]['context']['course_id'],data1[i]['time'].split("T")[0]))
    except:
        continue

NameError: name 'data1' is not defined

In [45]:
a=[]
for i in usertime:
    if i not in a:
        a.append(i)
    else:
        pass
target = {}
for user in uid:
    count = 0
    for i in range(len(a)):
        if a[i][0] == int(user):
            count +=1
        else:
            pass
    target[user]=count

In [66]:
#登入天數
#print(target)
target[661]

5

In [71]:
#Q2
video = []
for i in range(len(data1)):
    try:
        video.append((data1[i]['context']['user_id'],json.loads(data1[i]['event'])['id'],json.loads(data1[i]['event'])['currentTime']))
    except:
        pass

In [72]:
target_video = {}
for user in uid:
    count = 0
    videoid = []
    for i in range(len(video)):
        if video[i][0] == int(user):
            if video[i][1] not in videoid:
                count +=1
            videoid.append(video[i][1])
        else:
            pass
    target_video[user]=count

In [ ]:
## 觀看影片數量
print(target_video)
# target_video[661]

In [70]:
train['user_id'].value_counts()

2115     57
11327    19
17100    19
113      13
753      12
27       12
12960    12
15664    12
76        9
10367     9
8545      9
47        8
7228      8
34468     8
103       7
5362      7
30515     7
6752      6
2555      6
773       6
8362      6
43        6
20810     6
931       6
7365      6
9838      5
11601     5
275       5
33875     5
34372     5
         ..
6494      1
33117     1
35164     1
29019     1
31066     1
26968     1
22870     1
33109     1
35156     1
29011     1
6502      1
570       1
27016     1
22645     1
33157     1
31106     1
27008     1
22910     1
33149     1
29051     1
31098     1
27000     1
8738      1
16757     1
35188     1
14706     1
10608     1
365       1
2412      1
32768     1
Name: user_id, dtype: int64

In [185]:
df.head()

,course_id,session,user_id,event_type,video_id,currenttime,time,date,time_c
1,ORG4638/C9081/D1465,59d4351c901807c5a4a7e8246746544d,661,play_video,i4x-ORG4638-C9081-video-4aab9b87e47045e9808ef6...,24.581,2014-12-14T04:17:40.957994+00:00,2014-12-14,04:17:40
2,ORG4638/C9081/D1465,59d4351c901807c5a4a7e8246746544d,661,/courses/ORG4638/C9081/D1465/xblock/i4x:;_;_OR...,i4x-ORG4638-C9081-video-4aab9b87e47045e9808ef6...,NaN,2014-12-14T04:17:45.577600+00:00,2014-12-14,04:17:45
3,ORG4638/C9081/D1465,59d4351c901807c5a4a7e8246746544d,661,pause_video,i4x-ORG4638-C9081-video-4aab9b87e47045e9808ef6...,29.221,2014-12-14T04:17:45.577954+00:00,2014-12-14,04:17:45
4,ORG4638/C9081/D1465,59d4351c901807c5a4a7e8246746544d,661,play_video,i4x-ORG4638-C9081-video-4aab9b87e47045e9808ef6...,29.221,2014-12-14T04:17:58.024618+00:00,2014-12-14,04:17:58
5,ORG4638/C5126/D1465,ba3ded4534f5be0de0c7ea2839f84db8,1211,/courses/ORG4638/C5126/D1465/xblock/i4x:;_;_OR...,i4x-ORG4638-C9081-video-4aab9b87e47045e9808ef6...,NaN,2014-12-14T04:18:24.602380+00:00,2014-12-14,04:18:24


In [16]:
df['date'] = df['time'].str.split('T').str[0]
df['time_c'] = df['time'].str.split('T').str[1].str.split('.').str[0]

/Users/andy/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/andy/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [131]:
df[df["course_id"]=='ORG4638/C9081/D1465']["video_id"].value_counts()

i4x-ORG4638-C9081-video-4aab9b87e47045e9808ef6ec55c66783    36014
Name: video_id, dtype: int64

In [13]:
course_list=[]
for i in dic:
    course_list.append(i)
course_list

['ORG3685/C3456/D4353',
 'ORG3685/C7590/D6433',
 'ORG3685/C2979/D5610',
 'ORG5441/C7825/D2453',
 'ORG9615/C3524/D8881',
 'ORG9615/C1168/D6289',
 'ORG9615/C8617/D8881',
 'ORG4638/C2976/D5944',
 'ORG4638/C2976/D5552',
 'ORG4638/C1465/D0293',
 'ORG4638/C5998/D7477',
 'ORG4638/C5998/D4398',
 'ORG4638/C0627/D1937',
 'ORG4638/C0627/D2671',
 'ORG4638/C3685/D0323',
 'ORG4638/C6077/D3704',
 'ORG9784/C7351/D0468',
 'ORG9784/C9006/D9981',
 'ORG0137/C7590/D9615',
 'ORG0137/C0417/D2453',
 'ORG0137/C3721/D9981',
 'ORG0137/C3721/D0468',
 'ORG0137/C9976/D3704',
 'ORG0137/C9976/D2671',
 'ORG2288/C5915/D4353',
 'ORG6039/C4792/D2453',
 'ORG6039/C6077/D4353',
 'ORG6937/C7514/D0468',
 'ORG6937/C7514/D2671',
 'ORG8928/C0576/D6587',
 'ORG0323/C4386/D1168',
 'ORG0323/C2487/D6492',
 'ORG0323/C4724/D6492',
 'ORG0323/C1172/D1168',
 'ORG6153/C3187/D4353',
 'ORG8070/C4574/D4353',
 'ORG1525/C7481/D4353',
 'ORG1525/C4792/D0468',
 'ORG1525/C8152/D9981',
 'ORG5441/C0147/D6039',
 'ORG4638/C8244/D6039',
 'ORG4638/C6521/

In [50]:
#all
all_dic={}
all_id=0
test=["ORG4638/C9081/D1465"]

for a in course_list[:5]:
    df1=df[df["course_id"]==a]
    print(a)
#     print(len(df1['user_id'].value_counts()))
#     print(a)
    video_total=len(df1["video_id"].value_counts())

    for users in dic[a]:
#         print(users)
        df2=df1[(df1["user_id"]==int(users))]
        video=len(df2["video_id"].value_counts())

        session=len(df2["session"].value_counts())
        date=len(df2["date"].value_counts())
        video_complete_rate = video/video_total if video_total != 0 else 0
        all_dic[all_id]={"user_id":users,"course_id":a,"video":video,"session":session,'date':date,'video_complete_rate':video_complete_rate}
        all_id+=1


ORG3685/C3456/D4353
ORG3685/C7590/D6433
ORG3685/C2979/D5610
ORG5441/C7825/D2453
ORG9615/C3524/D8881


In [177]:
for a in (df['user_id']):
    print(type(int(a)))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

In [186]:
dic.keys()

dict_keys(['ORG3685/C3456/D4353', 'ORG3685/C7590/D6433', 'ORG3685/C2979/D5610', 'ORG5441/C7825/D2453', 'ORG9615/C3524/D8881', 'ORG9615/C1168/D6289', 'ORG9615/C8617/D8881', 'ORG4638/C2976/D5944', 'ORG4638/C2976/D5552', 'ORG4638/C1465/D0293', 'ORG4638/C5998/D7477', 'ORG4638/C5998/D4398', 'ORG4638/C0627/D1937', 'ORG4638/C0627/D2671', 'ORG4638/C3685/D0323', 'ORG4638/C6077/D3704', 'ORG9784/C7351/D0468', 'ORG9784/C9006/D9981', 'ORG0137/C7590/D9615', 'ORG0137/C0417/D2453', 'ORG0137/C3721/D9981', 'ORG0137/C3721/D0468', 'ORG0137/C9976/D3704', 'ORG0137/C9976/D2671', 'ORG2288/C5915/D4353', 'ORG6039/C4792/D2453', 'ORG6039/C6077/D4353', 'ORG6937/C7514/D0468', 'ORG6937/C7514/D2671', 'ORG8928/C0576/D6587', 'ORG0323/C4386/D1168', 'ORG0323/C2487/D6492', 'ORG0323/C4724/D6492', 'ORG0323/C1172/D1168', 'ORG6153/C3187/D4353', 'ORG8070/C4574/D4353', 'ORG1525/C7481/D4353', 'ORG1525/C4792/D0468', 'ORG1525/C8152/D9981', 'ORG5441/C0147/D6039', 'ORG4638/C8244/D6039', 'ORG4638/C6521/D8070', 'ORG4638/C8166/D6039', 

In [51]:
a = pd.DataFrame(all_dic).T

In [59]:
a.merge(train,how = 'right',on=('user_id','course_id'))

,course_id,date,session,user_id,video,video_complete_rate,target
0,ORG3685/C3456/D4353,NaN,NaN,51,NaN,NaN,F
1,ORG3685/C3456/D4353,NaN,NaN,2115,NaN,NaN,T
2,ORG3685/C3456/D4353,NaN,NaN,8362,NaN,NaN,F
3,ORG3685/C3456/D4353,NaN,NaN,17100,NaN,NaN,F
4,ORG3685/C3456/D4353,NaN,NaN,19452,NaN,NaN,T
5,ORG3685/C3456/D4353,NaN,NaN,21734,NaN,NaN,F
6,ORG3685/C3456/D4353,NaN,NaN,24224,NaN,NaN,F
7,ORG3685/C3456/D4353,NaN,NaN,25102,NaN,NaN,F
8,ORG3685/C3456/D4353,NaN,NaN,30993,NaN,NaN,F
9,ORG3685/C3456/D4353,NaN,NaN,32716,NaN,NaN,F


In [45]:
zip(a['user_id'],a['course'])
zip(df['user_id'],df['course_id'])

In [49]:
if zip(a['user_id'],a['course']) == zip(df['user_id'],df['course_id']):
    print('a')
else:
    print("a")

a


In [ ]:
df['str_user_id'] = df.to_string(columns = ['user_id'])

In [188]:
# video_count
test=["ORG9615/C8617/D8881"]
video_count=[]
for a in test:
    df1=df[df["course_id"]==a]
#     print(len(df1['user_id'].value_counts()))
#     print(a)
    for users in dic[a]:
#         print(users)
        df2=df1[(df1["user_id"]==users)]
    
        print(users,len(df2["video_id"].value_counts()))
#         video_count.append((users:len(df2["video_id"].value_counts())))

0


/Users/andy/anaconda/lib/python3.6/site-packages/pandas/core/ops.py:792: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [180]:
# session_count
test=["ORG4638/C9081/D1465"]
session_count=[]
for a in test:
    df1=df[df["course_id"]==a]
    print(len(df1['user_id'].value_counts()))
#     print(a)
    for users in dic[a]:
#         print(users)
        df2=df1[(df1["user_id"]==int(users))]
    
        print(users,len(df2["session"].value_counts()))

        session_count.append((users,len(df2["session"].value_counts())))

314
5 1
6 1
11 0
20 0
27 1
39 1
43 0
47 0
76 7
77 1
103 1
113 10
127 1
133 2
138 7
165 7
167 1
168 0
180 3
259 1
264 0
278 0
334 9
355 0
359 3
367 0
386 0
390 1
391 2
410 6
415 5
418 1
428 0
429 2
447 1
450 5
451 4
452 0
458 1
469 3
543 0
562 1
564 2
565 0
566 2
574 0
575 3
577 4
581 1
582 2
583 0
584 1
586 0
589 1
590 3
591 1
593 0
594 2
602 0
607 1
608 5
610 3
620 2
622 2
630 3
633 3
634 3
636 0
641 3
643 3
646 3
648 1
650 1
660 3
661 5
663 1
664 2
670 2
672 2
673 0
674 1
675 1
684 1
685 0
686 5
687 1
688 1
689 1
690 2
691 1
700 1
701 3
702 3
704 1
705 3
706 1
709 2
710 2
712 3
713 1
715 3
716 1
717 4
720 1
721 5
723 1
724 3
726 3
727 1
734 2
737 1
738 0
742 2
744 3
747 1
749 4
753 4
756 1
757 1
759 2
762 1
773 0
774 1
776 4
777 2
778 1
781 2
785 0
786 2
788 0
805 0
806 1
814 0
816 0
819 2
822 0
837 1
842 3
845 1
846 3
856 3
864 1
874 0
876 0
881 0
885 1
893 0
895 1
901 2
905 1
906 0
907 2
908 1
909 0
911 1
912 0
914 0
915 0
916 12
931 0
934 0
940 0
942 1
946 0
963 2
967 3
972 1
975 

In [8]:
# date_count
test=["ORG4638/C9081/D1465",
      "ORG3685/C3456/D4353"
     ]
date_count=[]
for a in test:
    df1=df[df["course_id"]==a]
    for users in dic[a]:
#         print(users)
        df2=df1[(df1["user_id"]==int(users))]
    
        print(users,len(df2["date"].value_counts()))

        date_count.append((users,len(df2["date"].value_counts())))

NameError: name 'df' is not defined

In [181]:
df2

,course_id,session,user_id,event_type,video_id,currenttime,time,date,time_c


In [ ]:
# _count
test=["ORG4638/C9081/D1465"]
session_count=[]
for a in test:
    df1=df[df["course_id"]==a]
    print(len(df1['user_id'].value_counts()))
#     print(a)
    for users in dic[a]:
#         print(users)
        df2=df1[(df1["user_id"]==int(users))]
    
        print(users,len(df2["session"].value_counts()))
        if len(df2["session"].value_counts()) == 0:
            pass
        else:
            session_count.append((users,len(df2["session"].value_counts())))

In [149]:
df1["video_id"].value_counts()

i4x-ORG4638-C9081-video-4aab9b87e47045e9808ef6ec55c66783    36014
Name: video_id, dtype: int64

In [128]:
video_count

[('5', 1),
 ('6', 1),
 ('27', 1),
 ('39', 1),
 ('76', 1),
 ('77', 1),
 ('103', 1),
 ('113', 1),
 ('127', 1),
 ('133', 1),
 ('138', 1),
 ('165', 1),
 ('167', 1),
 ('180', 1),
 ('259', 1),
 ('334', 1),
 ('359', 1),
 ('390', 1),
 ('391', 1),
 ('410', 1),
 ('415', 1),
 ('418', 1),
 ('429', 1),
 ('447', 1),
 ('450', 1),
 ('451', 1),
 ('458', 1),
 ('469', 1),
 ('562', 1),
 ('564', 1),
 ('566', 1),
 ('575', 1),
 ('577', 1),
 ('581', 1),
 ('582', 1),
 ('584', 1),
 ('589', 1),
 ('590', 1),
 ('591', 1),
 ('594', 1),
 ('607', 1),
 ('608', 1),
 ('610', 1),
 ('620', 1),
 ('622', 1),
 ('630', 1),
 ('633', 1),
 ('634', 1),
 ('641', 1),
 ('643', 1),
 ('646', 1),
 ('648', 1),
 ('650', 1),
 ('660', 1),
 ('661', 1),
 ('663', 1),
 ('664', 1),
 ('670', 1),
 ('672', 1),
 ('674', 1),
 ('675', 1),
 ('684', 1),
 ('686', 1),
 ('687', 1),
 ('688', 1),
 ('689', 1),
 ('690', 1),
 ('691', 1),
 ('700', 1),
 ('701', 1),
 ('702', 1),
 ('704', 1),
 ('705', 1),
 ('706', 1),
 ('709', 1),
 ('710', 1),
 ('712', 1),
 ('713'

In [159]:
ff=pd.DataFrame.from_dict(all_dic)

,0,1,2,3,4,5,6,7,8,9,...,542,543,544,545,546,547,548,549,550,551
course,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,...,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465,ORG4638/C9081/D1465
date,2,1,0,0,1,1,0,0,7,1,...,0,0,0,0,0,0,0,0,0,0
session,1,1,0,0,1,1,0,0,7,1,...,0,0,0,0,0,0,0,0,0,0
user,5,6,11,20,27,39,43,47,76,77,...,5804,5888,5898,5904,5912,5918,6024,7228,12221,12225
video,1,1,0,0,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
video_complete_rate,1,1,0,0,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [3]:
alldf = pd.read_csv('all.csv')

/Users/andy/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
len(alldf['course_id'].value_counts())

62

In [11]:
train

,course_id,user_id,target
0,ORG3685/C3456/D4353,51,F
1,ORG3685/C3456/D4353,2115,T
2,ORG3685/C3456/D4353,8362,F
3,ORG3685/C3456/D4353,17100,F
4,ORG3685/C3456/D4353,19452,T
5,ORG3685/C3456/D4353,21734,F
6,ORG3685/C3456/D4353,24224,F
7,ORG3685/C3456/D4353,25102,F
8,ORG3685/C3456/D4353,30993,F
9,ORG3685/C3456/D4353,32716,F
